In [406]:
import json
import pandas as pd
from clickhouse_driver import Client

In [407]:
CHUNK_SIZE = 100

In [408]:
client = Client(host="0.0.0.0")

In [409]:
client.execute("SHOW DATABASES")

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [410]:
client.execute("CREATE DATABASE IF NOT EXISTS test_db")

[]

In [411]:
client.execute("CREATE TABLE IF NOT EXISTS test_db.test_table (Id UInt64, EmployeeName String, \
                JobTitle FixedString(32), BasePay String, OvertimePay String, OtherPay String, \
                Benefits String, TotalPay Float64, TotalPayBenefits Float64, Year UInt8, \
                Notes String, Agency String, Status String) Engine=MergeTree() ORDER BY Id")

[]

In [412]:
client.execute("SHOW TABLES FROM test_db")

[('test_table',)]

In [506]:
df = pd.read_csv("./data/sf_salaries.csv", delimiter=",", encoding="utf-8", low_memory=False)

In [507]:
payload = json.loads(df.to_json(orient="records"))

In [508]:
data = (_ for _ in payload)

In [509]:
for i in range(len(payload)):
    print(data.__next__())
# client.execute(f"INSERT INTO test_db.test_table FORMAT JSONEachRow {payload}")
#for i in payload:
    # gen = dict(next(data))
    # result = gen.values()
    # for_load = (tuple(result)
    # print(i)
    # client.execute(f"INSERT INTO test_db.test_table (Id, EmployeeName, JobTitle, BasePay, OvertimePay, OtherPay, Benefits, TotalPay, TotalPayBenefits, Year, Agency, Status) VALUES {for_load}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'Id': 109224, 'EmployeeName': 'Charles G Fritz', 'JobTitle': 'Clinical Psychologist', 'BasePay': '2079.35', 'OvertimePay': '0.0', 'OtherPay': '0.0', 'Benefits': '20.79', 'TotalPay': 2079.35, 'TotalPayBenefits': 2100.14, 'Year': 2013, 'Notes': None, 'Agency': 'San Francisco', 'Status': None}
{'Id': 109225, 'EmployeeName': 'Chris  Ashley', 'JobTitle': 'Junior Clerk', 'BasePay': '1573.93', 'OvertimePay': '505.41', 'OtherPay': '0.0', 'Benefits': '20.79', 'TotalPay': 2079.34, 'TotalPayBenefits': 2100.13, 'Year': 2013, 'Notes': None, 'Agency': 'San Francisco', 'Status': None}
{'Id': 109226, 'EmployeeName': 'Jacqueline S Soriano', 'JobTitle': 'Public Svc Aide-Public Works', 'BasePay': '2078.0', 'OvertimePay': '0.0', 'OtherPay': '0.0', 'Benefits': '20.78', 'TotalPay': 2078.0, 'TotalPayBenefits': 2098.78, 'Year': 2013, 'Notes': None, 'Agency': 'San Francisco', 'Status': None}
{'Id': 109227, 'EmployeeName': 'Juan Carlo M Monton', 'JobTitle': 'Junior Clerk', 'BasePay': '1578.55', 'OvertimePay': 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

